In [1]:
import sys
import json
params = {'ptMetrics':True}
default = {'scenario': 'test', 'training_folder':'../..', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [2]:
# Add path to quetzal
sys.path.insert(0,'../../../../quetzal/')
from quetzal.model import stepmodel
import os
import numba as nb
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS

In [3]:
scenario = argv['scenario']
training_folder = argv['training_folder']
# if local. add the path to the scenario scenarios/<scenario>/
if on_lambda:
    input_folder = training_folder
else:
    input_folder = f'../scenarios/{scenario}/'
parallel_folder = os.path.join(input_folder,'parallel')
if not os.path.exists(parallel_folder):
    os.makedirs(parallel_folder)

In [4]:
def add_metrics_to_rlinks(links, rlinks):
    # add metrics to road links

    from quetzal.io.quenedi import split_quenedi_rlinks, merge_quenedi_rlinks

    rlinks = split_quenedi_rlinks(rlinks)

    df = links[['trip_id','route_id','headway','road_link_list']].explode('road_link_list')
    df['frequency'] = 1/(df['headway']/3600)

    agg_dict = {'trip_id':'nunique','route_id':'nunique','headway':lambda x: 1 / sum(1/x),'frequency':'sum'}
    df = df.groupby('road_link_list').agg(agg_dict)

    rename_dict = {'trip_id':'trip_id_count',
                'route_id':'route_id_count',
                'headway':'combine_headway (secs)',
                'frequency':'combine_frequency (veh/h)'}
    df = df.rename(columns=rename_dict)
    new_columns = df.columns
    new_columns = [col for col in new_columns if col not in rlinks.columns]
    rlinks = rlinks.merge(df,left_index=True,right_index=True,how='left')
    #cols_to_fill = [col for col in df.columns if col != 'combine_headway']
    #rlinks[cols_to_fill] = rlinks[cols_to_fill].fillna(0)

    rlinks = merge_quenedi_rlinks(rlinks,new_cols=new_columns)
    return rlinks

In [5]:
add_pt_metrics = argv['params'].get('ptMetrics',True)

In [6]:
import geopandas as gpd
import pandas as pd
from quetzal.engine.add_network_mapmatching import duplicate_nodes
from quetzal.io.quenedi import read_geojson

In [14]:
from syspy.spatial.utils import get_acf_distance

links_concat = []; nodes_concat = []
for name in os.listdir(parallel_folder):
    if name.startswith('links') and name.endswith('.geojson'):
        links_concat.append( read_geojson(os.path.join(parallel_folder, name)) )
    elif name.startswith('nodes') and name.endswith('.geojson'):
        nodes_concat.append( read_geojson(os.path.join(parallel_folder, name)) )

links = pd.concat(links_concat)
nodes = pd.concat(nodes_concat)

Skipping field road_link_list: unsupported OGR type: 5


In [11]:
if add_pt_metrics:
    road_links = gpd.read_file(os.path.join(input_folder,'road_links.geojson'), engine='pyogrio')
    road_links.set_index('index',inplace=True)
    road_links = add_metrics_to_rlinks(links, road_links)
    road_links.to_file(os.path.join(input_folder,f'road_links.geojson'), driver='GeoJSON', engine='pyogrio')

In [19]:
links['road_link_list'] = links['road_link_list'].fillna('[]')
links['road_link_list'] = links['road_link_list'].astype(str)
links.to_file(os.path.join(input_folder,f'links_final.geojson'), driver='GeoJSON')
nodes.to_file(os.path.join(input_folder,f'nodes_final.geojson'), driver='GeoJSON')


df = links.reset_index()[['index','trip_id']].copy()
df['acf_distance'] = links['geometry'].apply(lambda x: get_acf_distance([x.coords[0],x.coords[-1]],True))
df['routing_distance'] = links['length']
df['routing - acf'] = df['routing_distance']-df['acf_distance']
if 'shape_dist_traveled' in links.columns:
    df['shape_dist_traveled'] = links['shape_dist_traveled']
    df['routing - sdt'] = df['routing_distance']-df['shape_dist_traveled']

df2 = df.groupby('trip_id').agg(sum)
df2['routing - acf'] = df2['routing_distance']-df2['acf_distance']
if 'shape_dist_traveled' in df2.columns:
    df2['routing - sdt'] = df2['routing_distance']-df2['shape_dist_traveled']
df.to_csv(os.path.join(input_folder,'links_distances.csv'))
df2.to_csv(os.path.join(input_folder,'trips_distances.csv'))

